<a href="https://colab.research.google.com/github/queupe/criptologia/blob/main/Criptologia_aula02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Criptologia**


**Laboratório de criptologia**

Prof. Dr. Vilc Queupe **Rufino**

# **Aula 2**

** Cifras Simétricas AES e Funções Hash**

In [ ]:
# Biblioteca a ser instalada para realização das atividades
# Deve ser recarregada a cada instância Colab
!pip install pycryptodome

     |████████████████████████████████| 2.0 MB 5.4 MB/s 


**Operando com cifras AES**

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

#data = b'secret data'
data = b'the quick brown fox jumps over the lazy dog THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG'
key = get_random_bytes(16)

cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
txt = ''
with open("encrypted.bin", 'wb') as f:
  for x in (cipher.nonce, tag, ciphertext):
    f.write(x)
    txt += x.hex()

print(f"Texto legível: {len(data):03d} : {data}")
print(f"Texto claro  : {len(data):03d} : {data.hex()}" )
print(f"Texto cifrado: {len(txt)//2:03d} : {txt.upper()}")


Texto legível: 087 : b'the quick brown fox jumps over the lazy dog THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG'
Texto claro  : 087 : 74686520717569636b2062726f776e20666f78206a756d7073206f76657220746865206c617a7920646f672054484520515549434b2042524f574e20464f58204a554d5053204f56455220544845204c415a5920444f47
Texto cifrado: 119 : B10F2EFFC7406361F7C146920FAECFB845DB014F8F68E9F90A5245B5B5DD0AA884A14391837332169B4FFF70378CFB13A4A7B0D74A629EACB27F41EAE5F32F6456A6E0B2448BE8939AB288EC19FB53EABF978CF0D6CF14A068AF5DF31956A265BB2DE9CEAE376003F04E809BBEF00ED73E0651D3DEE0BE


In [ ]:
from Crypto.Cipher import AES

with open("encrypted.bin", "rb") as f:
  nonce, tag, ciphertext = [ f.read(x) for x in (16, 16, -1) ]

txt = nonce.hex() + tag.hex() + ciphertext.hex()

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)

print(f"Texto cifrado: {len(txt)//2:03d} : {txt.upper()}")
print(f"Texto claro  : {len(data):03d} : {data.hex()}" )
print(f"Texto legível: {len(data):03d} : {data}")

Texto cifrado: 119 : B10F2EFFC7406361F7C146920FAECFB845DB014F8F68E9F90A5245B5B5DD0AA884A14391837332169B4FFF70378CFB13A4A7B0D74A629EACB27F41EAE5F32F6456A6E0B2448BE8939AB288EC19FB53EABF978CF0D6CF14A068AF5DF31956A265BB2DE9CEAE376003F04E809BBEF00ED73E0651D3DEE0BE
Texto claro  : 087 : 74686520717569636b2062726f776e20666f78206a756d7073206f76657220746865206c617a7920646f672054484520515549434b2042524f574e20464f58204a554d5053204f56455220544845204c415a5920444f47
Texto legível: 087 : b'the quick brown fox jumps over the lazy dog THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG'


In [ ]:
# Sabendo-se que é possível definir a chave como:
key = b'\x74\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f'
print(f"Key: {len(key):03d} : {key.hex()}")

key = b'\0'*16
print(f"Key: {len(key):03d} : {key.hex()}")

Key: 016 : 740102030405060708090a0b0c0d0e0f
Key: 016 : 00000000000000000000000000000000


Exercícios:
1 - Use o AES nos modos 

Texto cifrado: 043 : 96CC3C79553787F6647CC28ACBABAF8B0DA7B7AB82D99E0F273AF1834770B371F40433BF4066EE9628B504
Texto claro  : 011 : 7365637265742064617461
Texto legível: 011 : b'secret data'


In [ ]:
#@title
# Implementação obtida em:
# https://github.com/rodrigorincon/criptografia/blob/master/AES/python/aesCipher.py

# -*- coding: UTF-8 -*-

def transformToMatriz(mensagem):
	matriz = [[],[],[],[]]
	for i in range(4):
		linha = [ord(caracter) for caracter in mensagem[i*4:i*4+4]]
		matriz[i] = linha
	return matriz


def transformToString(matriz):
	result = []
	for linha in range(4):
		result += [''.join( chr(i) for i in matriz[linha] )]
	return ''.join(result)

def sbox(valor):
	matriz_substituicao = [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
						   0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
						   0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
						   0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
						   0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
						   0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
						   0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
						   0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
						   0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
						   0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
						   0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
						   0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
						   0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
						   0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
						   0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
						   0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16]
	return matriz_substituicao[valor]

def invSbox(valor):
	matriz_substituicao = [0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
				   		   0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
						   0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
						   0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
						   0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
						   0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
						   0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
						   0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
						   0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
						   0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
						   0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
						   0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
						   0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
						   0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
						   0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
						   0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D]
	return matriz_substituicao[valor]

def rcon(j):
	vector = [[1,0,0,0],[2,0,0,0],[4,0,0,0],[8,0,0,0],[16,0,0,0],[32,0,0,0],[64,0,0,0],[128,0,0,0],[27,0,0,0],[54,0,0,0]]
	return vector[j]

def shiftLeft(linha, casas_deslocadas):
	return linha[casas_deslocadas:] + linha[:casas_deslocadas]

def funcaoG(linha, round):
	#shift pra esquerda
	linha = shiftLeft(linha,1)
	#sbox
	for i in range(4):
		linha[i] = sbox(linha[i])
	#Rcon(j)
	return xorLinha(linha,rcon(round))

def xorLinha(linha1, linha2):
	return [ linha1[0]^linha2[0], linha1[1]^linha2[1], linha1[2]^linha2[2], linha1[3]^linha2[3] ]

def keyExpansion(key):
	expanded = [key]
	for round in range(1,11):
		matriz = []
		for i in range(4):
			linha_anterior = funcaoG(expanded[-1][-1], round-1) if i==0 else matriz[-1]
			linha = xorLinha(linha_anterior, expanded[-1][i])
			matriz += [linha]
		expanded+= [matriz]
	return expanded

def addRoundKey(text, key):
	for i in range(4):
		text[i] = xorLinha(text[i],key[i])
	return text

def subBytes(text):
	for linha in range(4):
		for coluna in range(4):
			text[linha][coluna] = sbox(text[linha][coluna])
	return text

def shiftRow(text):
	linha1 = [text[0][0], text[1][1], text[2][2], text[3][3] ]
	linha2 = [text[1][0], text[2][1], text[3][2], text[0][3] ]
	linha3 = [text[2][0], text[3][1], text[0][2], text[1][3] ]
	linha4 = [text[3][0], text[0][1], text[1][2], text[2][3] ]
	return [linha1,linha2,linha3,linha4]

def xorCondicional(valor):
	return valor ^ 283 if valor > 255 else valor

def mult2(valor):
	return xorCondicional (valor << 1)
def mult4(valor): 
	return mult2( mult2(valor) )
def mult8(valor):
	return mult2( mult2(mult2(valor) ) ) 

def mult(coef, valor):
	value_returned = 0
	if(coef == 1):
		value_returned = valor
	elif(coef == 2):
		value_returned = xorCondicional(valor << 1)
	elif(coef == 3):
		value_returned = xorCondicional(valor << 1) ^ valor
	elif(coef==9):
		value_returned = mult8(valor) ^ valor
	elif(coef==11):
		value_returned = mult8(valor) ^ mult2(valor) ^ valor
	elif(coef == 13):
		value_returned = mult8(valor) ^ mult4(valor) ^ valor
	elif(coef == 14):
		value_returned = mult8(valor) ^ mult4(valor) ^ mult2(valor)
	return value_returned

def calcElement(coef, matriz, linha, coluna):
	return mult( coef[linha][0], matriz[coluna][0] ) ^ mult( coef[linha][1], matriz[coluna][1] ) ^ mult( coef[linha][2], matriz[coluna][2] ) ^ mult( coef[linha][3], matriz[coluna][3] )

def mixColumns(matriz):
	coef = [ [2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2] ]
	matriz_returned = []
	for coluna in range(4):
		linha = []
		for num_linha in range(4):
			linha += [calcElement(coef, matriz, num_linha, coluna)]
		matriz_returned += [linha]
	return matriz_returned

def initialTransformation(text, key):
	return addRoundKey(text, key[0])

def aesRound(text, key, round):
	subst_text = subBytes(text)
	shift_text = shiftRow(subst_text)
	mix_text = mixColumns(shift_text)
	return addRoundKey(mix_text, key[round])

def lastRound(text, key):
	subst_text = subBytes(text)
	shift_text = shiftRow(subst_text)
	return addRoundKey(shift_text, key[10])


In [ ]:
#@title
def invMixColumns(text):
	coef = [ [14,11,13,9],[9,14,11,13],[13,9,14,11],[11,13,9,14] ]
	matriz_returned = []
	for coluna in range(4):
		linha = []
		for num_linha in range(4):
			linha += [calcElement(coef, text, num_linha, coluna)]
		matriz_returned += [linha]
	return matriz_returned

def invShiftRow(text):
	linha1 = [text[0][0], text[3][1], text[2][2], text[1][3] ]
	linha2 = [text[1][0], text[0][1], text[3][2], text[2][3] ]
	linha3 = [text[2][0], text[1][1], text[0][2], text[3][3] ]
	linha4 = [text[3][0], text[2][1], text[1][2], text[0][3] ]
	return [linha1,linha2,linha3,linha4]

def invSubBytes(text):
	for linha in range(4):
		for coluna in range(4):
			text[linha][coluna] = invSbox(text[linha][coluna])
	return text

def inverseInitialTransformation(text, key):
	return addRoundKey(text, key[10])

def inverseAesRound(text, key, round):
	shift_text = invShiftRow(text)
	subst_text = invSubBytes(shift_text)
	xored_text = addRoundKey(subst_text, key[round])
	return invMixColumns(xored_text)

def inverseLastRound(text, key):
	shift_text = invShiftRow(text)
	subst_text = invSubBytes(shift_text)
	return addRoundKey(subst_text, key[0])
 
def toBinary(a):
  l,m=[],[]
  for i in a:
    l.append(ord(i))
  for i in l:
    m.append(int(bin(i)[2:]))
  return m


In [ ]:
## =======================================================================
# Criptograr 
def aesCipher(text,key):
	#converte as entradas para matrizes
	text = transformToMatriz(text)
	key = transformToMatriz(key)
	#come�a o algoritmo de fato
	key_expanded = keyExpansion(key)
	text = initialTransformation(text, key_expanded)
	for i in range(1,10):
		text = aesRound(text, key_expanded, i)
	cipherMatriz = lastRound(text, key_expanded)
	return transformToString(cipherMatriz)

## =======================================================================
# Decriptograr
def aesDecipher(text,key):
	#converte as entradas para matrizes
	text = transformToMatriz(text)
	key = transformToMatriz(key)
	#come�a o algoritmo de fato
	key_expanded = keyExpansion(key)
	text = inverseInitialTransformation(text, key_expanded)
	for i in [9,8,7,6,5,4,3,2,1]:
		text = inverseAesRound(text, key_expanded, i)
	cipherMatriz = inverseLastRound(text, key_expanded)
	return transformToString(cipherMatriz)

In [ ]:
#        0123456789abcdef
data = b'secrets of night'
key  = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f'
iv   = b'\0'*16

text_cifrado = aesCipher(data.decode(), key.decode())

text_decifrado = aesDecipher(text_cifrado, key.decode())

print(f"Texto cifrado   : {len(text_cifrado):03d} : {str.encode(text_cifrado).hex()}")
print(f"Texto decifrado : {len(text_decifrado):03d} : {text_decifrado}" )


Texto cifrado   : 016 : 337b1a5cc3a27cc289091dc2a82c52c3aec38428c3b4
Texto decifrado : 016 : secrets of night


In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util import Counter

#        0123456789abcdef
data = b'secrets of nightsecrets of night'
key  = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f'
iv   = b'\0'*16
#iv   = b'\x74\x01\x02\x03\x32\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f'
ctr  = Counter.new(128)
ctr['initial_value'] =2
#Choose the cipher MODE:
mode = AES.MODE_CBC

if mode == AES.MODE_CBC:
  cipher   = AES.new(key, AES.MODE_CBC,iv)
  decipher = cipher
elif mode == AES.MODE_ECB:
  cipher  = AES.new(key, AES.MODE_ECB)
  decifer = cipher
elif mode == AES.MODE_CTR:
  cipher   = AES.new(key, AES.MODE_CTR, counter=ctr)
  print(ctr)

ciphertext2 = cipher.encrypt(data)


if mode == AES.MODE_CBC:
  decipher = AES.new(key, AES.MODE_CBC,iv)
elif mode == AES.MODE_ECB:
  decipher = AES.new(key, AES.MODE_ECB)
elif mode == AES.MODE_CTR:
  decipher = AES.new(key, AES.MODE_CTR, counter=ctr)
  print(ctr)
deciphertext2 = decipher.decrypt(ciphertext2)

print(f"Texto legível: {len(ciphertext2):03d} : {ciphertext2.hex()}")
print(f"Texto legível: {len(deciphertext2):03d} : {deciphertext2}")


Texto legível: 032 : 337b1a5ce27c89091da82c52eec428f4f7d092417806880c32b0dbef333ab279
Texto legível: 032 : b'secrets of nightsecrets of night'
